In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
model(model.dummy_inputs)  # Build the model
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
from google.colab import files
import pandas as pd
import io
uploaded = files.upload()

Saving Sample_Postings.csv to Sample_Postings.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['Sample_Postings.csv']))
df

,Unnamed: 0,Unnamed: 1
0,job_id,description
1,410199029,"Revenue Accountant Yoshinoya America, Inc. T..."
2,419647878,"GENERAL WAREHOUSE - Jackson, GA 30233 - Indeed..."
3,424561936,"Technical Consultant ServiceNow Santa Clara,..."
4,408897225,Workday & HR Data Integrations Product Owner ...
...,...,...
3994,407991284,This job has expired on Indeed Reasons could ...
3995,424765603,"Team Leader, PM, Bakery Publix Lakeland, FL ..."
3996,424439648,Preowned Sales Consultant Toyota of Irving I...
3997,411508888,This job has expired on Indeed Reasons could ...


In [ ]:
df.columns = ['job_id', 'description']
postings = df.iloc[1:, :]
postings

,job_id,description
1,410199029,"Revenue Accountant Yoshinoya America, Inc. T..."
2,419647878,"GENERAL WAREHOUSE - Jackson, GA 30233 - Indeed..."
3,424561936,"Technical Consultant ServiceNow Santa Clara,..."
4,408897225,Workday & HR Data Integrations Product Owner ...
5,443365857,"HR Systems Solution Lead - Aurora, IL 60504 - ..."
...,...,...
3994,407991284,This job has expired on Indeed Reasons could ...
3995,424765603,"Team Leader, PM, Bakery Publix Lakeland, FL ..."
3996,424439648,Preowned Sales Consultant Toyota of Irving I...
3997,411508888,This job has expired on Indeed Reasons could ...


In [ ]:
train_df = postings.iloc[0:3800,:]
eval_df = postings.iloc[3800:,:]
train_df.head()

,job_id,description
1,410199029,"Revenue Accountant Yoshinoya America, Inc. T..."
2,419647878,"GENERAL WAREHOUSE - Jackson, GA 30233 - Indeed..."
3,424561936,"Technical Consultant ServiceNow Santa Clara,..."
4,408897225,Workday & HR Data Integrations Product Owner ...
5,443365857,"HR Systems Solution Lead - Aurora, IL 60504 - ..."


In [ ]:
#Changing the format
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
type(train_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
#Tokenization

def tokenize_function(examples):
    result = tokenizer(examples["description"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# Use batched=True to activate fast multithreading!
train_tokenized_datasets = train_dataset.map(
    tokenize_function, batched=True, remove_columns=train_df.columns.tolist()
)

eval_tokenized_datasets = eval_dataset.map(
    tokenize_function, batched=True, remove_columns=eval_df.columns.tolist()
)

train_tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (885 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 3800
})

In [ ]:
chunk_size = 256
embedding_dims = 768
text_size = 140

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

train_mlm_datasets = train_tokenized_datasets.map(group_texts, batched=True)
eval_mlm_datasets = eval_tokenized_datasets.map(group_texts, batched=True)

train_mlm_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 13218
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15, return_tensors='tf')

tf_train_dataset = train_mlm_datasets.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

# Note we are not shuffling the eval dataset.
tf_eval_dataset = eval_mlm_datasets.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

18/18 [==============================] - 11s 391ms/step - loss: 3.2428
Perplexity: 25.60


In [ ]:
model.fit(tf_train_dataset)

414/414 [==============================] - 502s 1s/step - loss: 2.8458


In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

18/18 [==============================] - 7s 402ms/step - loss: 2.4628
Perplexity: 11.74


In [ ]:
text = "Responsible for product performance Perform coding assignments Manage Sprint Cycles Review codes Lead tech discovery in data, social and link analysis Conduct testing and debugging of codes Support team in development and [MASK] Provide supervision and guidance to developers "

In [ ]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> Responsible for product performance Perform coding assignments Manage Sprint Cycles Review codes Lead tech discovery in data, social and link analysis Conduct testing and debugging of codes Support team in development and to Provide supervision and guidance to developers 
>>> Responsible for product performance Perform coding assignments Manage Sprint Cycles Review codes Lead tech discovery in data, social and link analysis Conduct testing and debugging of codes Support team in development and development Provide supervision and guidance to developers 
>>> Responsible for product performance Perform coding assignments Manage Sprint Cycles Review codes Lead tech discovery in data, social and link analysis Conduct testing and debugging of codes Support team in development and help Provide supervision and guidance to developers 
>>> Responsible for product performance Perform coding assignments Manage Sprint Cycles Review codes Lead tech discovery in data, social and link analysis Con

In [ ]:
#### Saving ###
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/

# trainer.save_pretrained ("Trained model") #assumed this would save but did not
model.save_pretrained ("Trained JobDesc model") #did save

Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/gdrive/My Drive/FOLDER'
/content


In [ ]:
%cd /content/gdrive/My\ Drive/Colab Notebooks/Pre-training

/content/gdrive/My Drive/Colab Notebooks/Pre-training


In [ ]:
model.save_pretrained ("Trained JobDesc model") #did save

In [ ]:
model_loaded = TFAutoModelForMaskedLM.from_pretrained("Trained JobDesc model", local_files_only=True)
model_loaded 

All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at Trained JobDesc model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
model_loaded.summary()

Model: "tf_distil_bert_for_masked_lm_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
________________________

In [ ]:
num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model_loaded.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:

eval_loss = model_loaded.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

18/18 [==============================] - 6s 197ms/step - loss: 2.3806
Perplexity: 10.81
